In [1]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir('drive/My Drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#!tar -zxvf aff_wild_videos_annotations_bboxes_landmarks.tar.gz
#os.chdir('/content/drive/My Drive/data_aff_wild/landmarks/')

#!tar -zxvf test.tar.gz


test/103/454.pts
test/267/2232.pts
test/381/381.pts
test/119/2800.pts
test/244/52.pts
test/281/6804.pts
test/336/1893.pts
test/241/498.pts
test/194/3161.pts
test/204/3920.pts
test/241/1168.pts
test/147/12226.pts
test/307/1427.pts
test/147/6864.pts
test/274/1044.pts
test/139/2734.pts
test/103/18576.pts
test/204/3.pts
test/268/2173.pts
test/197/42.pts
test/119/436.pts
test/357/732.pts
test/103/5527.pts
test/165/1539.pts
test/268/3376.pts
test/139/3937.pts
test/375/798.pts
test/304/1737.pts
test/367/673.pts
test/434/2711.pts
test/322/447.pts
test/416/4909.pts
test/159/1097.pts
test/265/2483.pts
test/380/691.pts
test/199/1356.pts
test/450/5594.pts
test/450/1811.pts
test/102/764.pts
test/147/990.pts
test/284/2476.pts
test/307/1583.pts
test/103/4480.pts
test/416/3862.pts
test/147/5049.pts
test/103/8410.pts
test/119/592.pts
test/304/1893.pts
test/265/3686.pts
test/103/10863.pts
test/450/6797.pts
test/165/1695.pts
test/209/880.pts
test/311/816.pts
test/284/3679.pts
test/281/2786.pts
^C


In [2]:
DATA_DIR = "data_aff_wild"
ANNOTATION_TRAIN_DIR = os.path.join(DATA_DIR, "annotations/train")
AROUSAL_TRAIN_DIR = os.path.join(DATA_DIR, "annotations/train/arousal")
VALENCE_TRAIN_DIR = os.path.join(DATA_DIR, "annotations/train/valence")
AROUSAL_TRAIN_FILES = os.path.join(AROUSAL_TRAIN_DIR, "*.txt")
VALENCE_TRAIN_FILES = os.path.join(VALENCE_TRAIN_DIR, "*.txt")

#BBOXES_TRAIN_DIR = os.path.join(DATA_DIR, "bboxs/train")
#LANDMARKS_TRAIN_DIR = os.path.join(DATA_DIR, "landmarks/train")
VIDEOS_TRAIN_DIR = os.path.join(DATA_DIR, "videos/train")
VIDEOS_TEST_DIR = os.path.join(DATA_DIR, "videos/test")

In [3]:
"""
Sanity: if the number of annotations same as the number of frames.
"""
import os

import numpy as np
import cv2
os.chdir('/content/drive/My Drive/')


def count_frame(path: str) -> int:
    video_obj = cv2.VideoCapture(path)
    frame_counter = 0
    success = 1
    while success:
        success, _ = video_obj.read()
        frame_counter += 1
    return frame_counter - 1

def count_lines(path: str) -> int:
    line_counter = 0
    with open(path) as f:
        for _ in f:
            line_counter += 1
    return line_counter
num_frames = count_frame(os.path.join(VIDEOS_TRAIN_DIR, "105.avi"))
num_annotations = count_lines(os.path.join(AROUSAL_TRAIN_DIR, "105.txt"))
print(num_frames)
print(num_annotations)
assert num_frames == num_annotations

10076
10076


In [6]:
"""
Filter frames by arousal and valence and save to a folder.
"""
import glob
from collections import defaultdict

import cv2

BORED = (-0.34, -0.78)
DROOPY = (-0.32, -0.96)
TIRED = (-0.02, -0.99)
SLEEPY = (0.02, -0.99)
EMOTIONS = [BORED, DROOPY, TIRED, SLEEPY]

def is_bored(valence: float, 
             arousal: float, 
             radius: float=0.1) -> bool:
    """
    Defined bored in valence, arousal. See Fig. 1 in https://arxiv.org/pdf/1804.10938.pdf.
    """
    is_bored = False
    for (emotion_valence, emotion_arousal) in EMOTIONS:
        with_in_radius = (emotion_valence - radius <= valence <= emotion_valence + radius and 
            emotion_arousal - radius <= arousal <= emotion_arousal + radius)
        is_bored = is_bored or with_in_radius
    return is_bored

def get_video_path(video_id: str):
    mp4_path = os.path.join(VIDEOS_TRAIN_DIR, video_id + ".mp4")
    avi_path = os.path.join(VIDEOS_TRAIN_DIR, video_id + ".avi")
    if os.path.isfile(mp4_path):
        return mp4_path
    if os.path.isfile(avi_path):
        return avi_path
    raise ValueError("Video {} does not exist".format(video_id))

def create_candidates(output_dir="candidates", radius: float=0.2) -> str:
    candidate_frames = defaultdict(set)
    total_count = 0
    bored_count = 0
    for arousal_file, valence_file in zip(sorted(glob.glob(AROUSAL_TRAIN_FILES)), sorted(glob.glob(VALENCE_TRAIN_FILES))):
        video_id = os.path.splitext(os.path.basename(arousal_file))[0] # eg. file_id = 105
        arousal_file = open(arousal_file, 'r')
        valence_file = open(valence_file, 'r')
        arousal_values = [float(line) for line in arousal_file.readlines()]
        valence_values = [float(line) for line in valence_file.readlines()]
        for frame_id, (valence_value, arousal_value) in enumerate(zip(valence_values, arousal_values)):
            total_count += 1
            if is_bored(valence_value, arousal_value, radius=radius):
                bored_count += 1
                candidate_frames[video_id].add(frame_id)

    # Create frame file for each candidate frames.
    output_dir_bored = os.path.join(output_dir, str(radius), "non_productive")
    output_dir_non_bored = os.path.join(output_dir, str(radius), "productive")
    
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(output_dir_bored, exist_ok=True)
    os.makedirs(output_dir_non_bored, exist_ok=True)
    for video_id, candidate_frames in candidate_frames.items():
        video_file = get_video_path(video_id)
        video_obj = cv2.VideoCapture(video_file)
        frame_id = 0
        success = True
        while success:
            success, frame = video_obj.read()
            if success:
                if frame_id in candidate_frames:
                    cv2.imwrite(os.path.join(output_dir_bored, "{}_{}.jpg".format(video_id, frame_id)),
                                frame)
                else:
                    cv2.imwrite(os.path.join(output_dir_non_bored, "{}_{}.jpg".format(video_id, frame_id)),
                                frame)
            frame_id += 1

    return candidate_frames, bored_count, total_count
    
    
candidate_frames, bored_count, total_count = create_candidates()

In [ ]:
"""
Radius = 0.3, annotation. Inclusive range.
"""
annotations = [
    ["299_448", "299_454"],
    ["305_1712", "305_1760"],
    ["308_247", "308_286"],
    ["312_1289", "312_1329"],
    ["313_324", "313_345"],
    ["313_1276", "313_1625"],
    ["369_680", "369_896"],
    ["377_2034", "377_2234"], 
    ["385_244", "385_521"],
    ["386_84", "386_128"],
    ["389_418", "389_3404"],
    ["400_42", "400_72"],
    ["447_929", "447_1118"],
    ["448_1295", "448_2571"]
]